In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-10-14 09:01:43--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2025-10-14 09:01:43 (189 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [91]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [87]:
# Drop NAs
df_books.dropna(inplace=True)
df_ratings.dropna(inplace=True)

# Merge ratings with book titles
df_merged = df_ratings.merge(df_books[['isbn', 'title']], on='isbn')

# Filter out users with less than 200 ratings and books with less than 100 ratings on the merged data
user_n_ratings_merged = df_merged['user'].value_counts()
book_n_ratings_merged = df_merged['title'].value_counts()

df_filtered = df_merged[(~df_merged['user'].isin(user_n_ratings_merged[user_n_ratings_merged < 200].index)) &
                        (~df_merged['title'].isin(book_n_ratings_merged[book_n_ratings_merged < 100].index))]


# User-item matrix
df_pivot = df_filtered.pivot_table(index='user', columns='title', values='rating').fillna(0).T

# Rename df_pivot to df_ratings for consistency with subsequent cells
df_ratings = df_pivot

In [50]:
print(df_ratings)

user                                                254     2276    2766    \
title                                                                        
1984                                                   9.0     0.0     0.0   
1st to Die: A Novel                                    0.0     0.0     0.0   
24 Hours                                               0.0     0.0     0.0   
2nd Chance                                             0.0    10.0     0.0   
4 Blondes                                              0.0     0.0     0.0   
...                                                    ...     ...     ...   
Year of Wonders                                        0.0     0.0     0.0   
You Belong To Me                                       0.0     0.0     0.0   
Zen and the Art of Motorcycle Maintenance: An I...     0.0     0.0     0.0   
Zoya                                                   0.0     0.0     0.0   
\O\" Is for Outlaw"                                    0.0     0

In [88]:
# Fit KNN model
model = NearestNeighbors(metric='cosine')
model.fit(df_ratings.values)

NearestNeighbors(metric='cosine')

In [90]:
# Check if book exists
book = 'Catch 22'
title = df_ratings.loc[book]

# Book index
book_index = df_ratings.index.get_loc(book)

# Get neighbors
distance, indices = model.kneighbors([title.values], n_neighbors=6)

#Recommendation list
recommended_books = []

for i in range(1, len(indices[0])):
      similar_index = indices[0][i]
      recommended_books.append([df_ratings.index[similar_index], float(distance[0][i])])

# Keep only top 5 recommendations
recommended_books = recommended_books[:5]

recommended_books = [book, recommended_books]

print(recommended_books)


['Catch 22', [['Saint Maybe', 0.7120952606201172], ['One Hundred Years of Solitude', 0.7131385803222656], ['Cryptonomicon', 0.7200307846069336], ['To the Lighthouse', 0.721048891544342], ['The Reptile Room (A Series of Unfortunate Events, Book 2)', 0.729436993598938]]]


In [47]:
print(df_ratings.index)

Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       ...
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
      dtype='object', name='title', length=29317)


In [83]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  if book not in df_ratings.index:
    return f" '{book}' does not exist in database."

  book_index = df_ratings.index.get_loc(book)

  distance, indices = model.kneighbors([title.values], n_neighbors=6)

  recommended_books = []

  for i in range(1, len(indices[0])):
      similar_index = indices[0][i]
      recommended_books.append([df_ratings.index[similar_index], float(distance[0][i])])

  # Keep only top 5 recommendations
  recommended_books = recommended_books[:5]
  recommended_books = [book, recommended_books]

  return recommended_books

In [84]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['Blue Diary', 0.7145698070526123], ['The Lovely Bones: A Novel', 0.715856671333313], ['I Know This Much Is True', 0.7496687769889832], ['Icy Sparks', 0.7646523118019104], ['The Weight of Water', 0.7678344249725342]]]
You haven't passed yet. Keep trying!
